# Text Analysis in Python 4: Topics & Emotions

<h1 style="text-align:center;font-size:300%;">The State of the Union is... scared??</h1> 
  <img src="https://api.time.com/wp-content/uploads/2016/07/emoticons.jpg?quality=85&w=600" style="width:%80;">
  <!--<img src="http://www.languagetrainers.com/blog/wp-content/uploads/2012/10/us-are-vs-us-is1.png" style="width:%140;">-->



## Topic Modeling, Sentiment Analysis, and other Inductive Approaches to Text Analysis

To what extent can text analysis approaches identify topics, emotions, and other aspects of texts without any prior input by the researcher?

In this session, participants will:
+ learn what sentiment analysis and topic modeling can and can't do
+ modify and interpret a series of sentiment analysis graphs
+ briefly experiment with topic modeling while examining its limits

<!--
*Explain Topic Modeling / Give Examples Of (both good and bad)
Present Buckets of Words Examples and have students interpret
Apply Topic Modeling to SOTU Texts (does it make sense to do so? it certainly would be interesting to see which SOTU addresses are lumped in with which others)
Compare to out-of-the-box approaches (Voyant, etc.)*-->

<div class="alert alert-success" role="alert"><h3 style="color:green">Code Together:</h3><p style="color:green">In these cell blocks, we will code together. You can find the completed version in our shared folder (ending with "_completed.ipynb").</p></div>

<div class="alert alert-info" role="alert"><h3 style="color:blue;">Exercises:</h3><p style="color:blue">are in blue text. These are a chance to practice what you have learned.</p></div>

<div style = "background-color:#f3e5f5"><h3 style="color:purple">Python Basics - Additional Practice</h3><p style="color:purple">are in purple text. Work on these after the lesson if you would like more practice.</p></div>

## Further Practice

For further practice, we recommend you review the sections on [Sentiment Analysis](https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/04-Sentiment-Analysis.html) and [Topic Modeling](https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/05-Topic-Modeling.html) in Ch. 5 in Melanie Walsh's *Introduction to Cultural Analytics*. You can even practice with this code yourself by pressing the Rocket Ship icon at the top of each section and opening a [**Binder**](https://mybinder.org/) session which will allow you to run code in her Jupyter Notebooks.

## Part I: Setup

1. First, let's import all required libraries/packages.

In [ ]:
import pathlib, os             #for navigating through files and working with file paths
from pathlib import Path
import nltk                    #Natural Language Toolkit - large package for text analysis and natural language processing
import spacy                   # more NLP
nlp = spacy.load("en_core_web_sm")

import re                      #re = regular expressions - a tool for searching for patterns in texts (i.e. remove all punctuation; identify all words that begin with a capitalized letter; identify phone numbers by searching for the pattern ###-###-####) 
import collections
import gensim                  # for topic modeling 
import pandas as pd            #for working with dataframes
import numpy as np             # for working with arrays of numbers (single dimension: array; 2d: matrix; 3+d: tensor)
import matplotlib              #these 2 lines of code: importing Python libraries for visualization
import matplotlib.pyplot as plt, seaborn as sns

from ast import literal_eval

nltk.download('punkt')         #nltk - here we need to download and import specific packages from nltk
from nltk import word_tokenize
from nltk import tokenize
from nltk.stem.snowball import SnowballStemmer

from nltk.stem import WordNetLemmatizer
nltk_lemmatizer = WordNetLemmatizer()


# from tqdm import tqdm_notebook as tqdm
#from tqdm import notebook as tqdm   #for creating a progress bar to show computation in progress
#from tqdm import trange

plt.style.use('fivethirtyeight')    # sets the style for our plots, other style options here: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 9]  #sets size of the visualizations in our notebook
#%config InteractiveShellApp.matplotlib = 'inline'
#%config InlineBackend.figure_formats = ['svg']




2. ... and then open our dataframe with the fulltext and tokenized lists for each SOTU address. 

In [ ]:
sotudf = pd.read_csv("sotudf.tsv", encoding = "utf-8", sep = "\t", index_col = 0, 
                     converters = {'tokens': literal_eval, 'ltoks': literal_eval, 'ltoks_ns': literal_eval})
#converters needed otherwise pandas reads in a list of tokens [gentlemen, of, the, ...] and surrounds each word with quotation marks
## like ['gentlemen', 'of', 'the', ...] which creates all kinds of problems
sotudf.head(3)

3. As you may recall, there is one empty text file in our dataset. Since Washington did not give a SOTU address in 1790, the person who compiled this SOTU corpus created an empty text file for 1790 to indicate that the 1790 address was not forgotten, but simply was never given.

For our sentiment analysis, however, we want to remove all null data (in this case the 1790 dummy file). We can do that using the **.notnull() function. When subsetting a dataframe, a simple way to see how it changed is to compare the dimensions of the dataframe (number of rows and columns) before and after applying the subset.

In [ ]:
print(sotudf.shape)
sotudf = sotudf[sotudf['fulltext'].notnull()]
print(sotudf.shape)  #after removing empty 1790 row

<div class="alert alert-success" role="alert">
<h3  style="color:green">Code Together</h3>

<p style="color:green"> 4. Using the <a href = "https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf">Python Pandas Cheat Sheet (click here)</a>, inspect our SOTU dataframe ("sotudf") retrieving the number of rows and columns and the types of data stored in each column.</p>

</div>

<div class="alert alert-success" role="alert">

<p style="color:green"> 5. Using the <a href = "https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf">Python Pandas Cheat Sheet (click here)</a>, can you identify if all lists of tokens are unique? (If they aren't then that indicates we made an error in compiling the dataset.)</p>

</div>

<div class="alert alert-success" role="alert">
<p style="color:green"> 6. Next, we will create two new columns:
<ol style="color:green">
    <li>one that calculates the number of characters (rather than words) for each SOTU address</li>
    <li>another that calculates the average word-length of each address</li>
</ol>
<p style="color:green"><i>To do this, we can call an anonymous function using <b>lambda</b> within the <b>.apply()</b> method. There are other ways that are faster</p>
</div>

## Part II. Sentiment Analysis - An Overview

7. **Sentiment Analysis** is a text analysis method that assigns emotional scores to segments of texts. 

The most common sentiment analysis tools assign words and phrases a sentiment score ranging from positive to negative. The VADER lexicon, for example, assign relies on a dataset of phrases coded by human readers between extremely positive (+4) and extremely negative (-4). You can learn more about VADER and practice with it in the [Sentiment Analysis](https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/04-Sentiment-Analysis.html) section in Melanie Walsh's *Cultural Analytics* ebook.

Other sentiment analysis tools can identify more nuanced emotions. For example, in this lesson, we will be using the [NRC Word-Emotion Association Lexicon](https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm). According to this lexicon's webpage:

```
The NRC Emotion Lexicon is a list of English words and their associations with eight basic emotions (anger, fear, anticipation, trust, surprise, sadness, joy, and disgust) and two sentiments (negative and positive). The annotations were manually done by crowdsourcing.
```

The code below, which uses the NRC, was borrowed and adapted from Greg Rafferty's sentiment analysis of the Harry Potter novels: https://github.com/raffg/harry_potter_nlp/blob/master/sentiment_analysis.ipynb.

## Part III: Pre-processing

8. We've already done a lot of the necessary pre-processing of our SOTU addresses (tokenizing, lower-casing, removing stopwords, etc.). Let's review what our SOTU dataframe looks like:

In [ ]:
sotudf.head(3)

9. We could run sentiment analysis on our lower-case tokens with stopwords removed (the column: "ltoks_ns"). However, what about distinguishing plural ("citizens", "people", "gentlemen") from singular forms ("citizen", "person", "gentleman") forms of nouns? What about different conjugations of the same verb ([be, am, is, are, was, were], [play, plays, played])?

To group together words that have a common root, we can *lemmatize* each word, that is convert it to its *lemma*. One way to do this is to use NLTK's **WordNetLemmatizer** to return the lemma of each word. Let's see how it does.

In [ ]:
words = ["play", "plays", "player", "players", "played", "playable", "rocks", "corpora", "better", "be", "am", "is", "were", "are", "break", "breaks", "broke", "broken", "citizen", "citizens", "women", "women",
         "cactus", "cacti", "disturb", "disturbs", "disturbing", "go", "goes", "went"]
[nltk_lemmatizer.lemmatize(word) for word in words]

10. Not perfect, but it does simplify our word list a little bit. For more advanced projects, you could try using **spaCy**'s lemmatizing functions that also read in the Part of Speech (POS) tag for each word. I.e., reading in ("scared", "ADJ") will produce different results than ("scared", "VERB").

But, for this project, NLTK's lemmatizer will work well enough:

In [ ]:
sotudf['lemmas'] = sotudf['ltoks_ns'].apply(lambda x: [nltk_lemmatizer.lemmatize(ltok) for ltok in x])
sotudf.head(3)

11. Let's create a frequency list of each lemma using **collections.Counter()**.

In [ ]:
sotudf['freqlemmas'] = sotudf['lemmas'].apply(lambda x: collections.Counter(x))
sotudf.head(3)

12. Now, let's subset our dataframe by only keeping those columns we need.

*Note: we could also subset our dataframe this way:*

```
sotusub = sotudf[['year', 'pres', 'numtoks', 'freqlemmas']]
```

*However, this could cause errors later, so it is better to create subsets using the **.loc[]** method.*


In [ ]:
sotusub = sotudf.loc[:,['year', 'pres', 'numtoks', 'freqlemmas']]
sotusub.head(3)

<div class="alert alert-info" role="alert" style='color:blue'><h3>Exercises for Part III</h3>

<p>13. Create a new column to store an all upper-case version of our fulltext strings in the 'full_text' column.</p>
</div>

## Part IV: Applying Sentiment Analysis to the SOTU Corpus

Our data is now ready for sentiment analysis.

The code below gets a little complicated. In essence it creates a function **corpus_emo_scores** that reads in a dataframe and column name (which identifies the column in the dataframe the frequency dictionary of lemmatized words from each text). It also reads in the NRC Emotions Lexicon that we will use to apply emotions scores to each of our texts.

It then calls the **emo_score_by_text** function, which scores each text for the eight emotions and two sentiments included in the NRC Emotions Lexicon. This in turn call the function **emo_score**.

14. Let's run the code below to place three new functions into memory. Then we will examine exactly how it works more closely.

**Note: Before running this, you will need to download the [NRC Emotion Lexicon](https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm).**



In [ ]:
def emo_score(lemma, freq, emolex_words):
    '''
    INPUT: lemmatized, lower-cased word (lemma) + frequency this lemma appears in a text
    OUTPUT: list of 10 NRC emotions scores (8 emotions + 2 sentiments [positive / negative])

    reads in a lemmatized word, looks it up in NRC lexicon dictionary (emolex_words) and returns the score of the word
    for the 8 emotions and 2 sentiments in the lexicon

    then takes the resulting list of 10 scores and multiples each by the frequency of the word
    returning the new list

    '''
    if (emolex_words[ 'word'].eq(lemma)).any():   #if any word in the "word" column in emolex_words equals the inputted lemma:
        #print(lemma)
        emolex_row = emolex_words[emolex_words['word'] == lemma].loc[:,'anger':'trust']
        #print(emolex_row)
        emolex_list = [item for item in emolex_row.iloc[0]]
        new_emo_scores = [item * freq for item in emolex_list]
        #print(new_emo_scores, "\n")
        #emo_scores = [x + y for x,y in zip(emo_scores, new_emo_scores)]
        return(new_emo_scores)
    else:                                          #else: lemma not in emolex_words dataframe
        return(None)

def emo_score_by_text (lemmadict, emolex_words):
    '''
    INPUT: reads in a frequency dictionary of lemmatized words (lemmadict)
    OUTPUT: returns a list of 10 cumulative scores for each of the 8 emotions and 2 sentiments in the NRC lexicon


    '''
    cum_emo_score = [0,0,0,0,0,0,0,0,0,0]
    for key, val in lemmadict.items():
        temp_emo_score = emo_score(key, val, emolex_words)
        if temp_emo_score is not None:
            cum_emo_score = [x + y for x, y in zip(cum_emo_score, temp_emo_score)]
    return(cum_emo_score)

def corpus_emo_scores(df, freq_lemmas_col):
    '''
    INPUT: a dataframe of texts (df), with a column name (freq_lemmas_col) indicating a column
    with a frequency dictionary of lemmatized words for each text
    OUTPUT: returns df with a new column recording the total 10 NRC scores for each text
    '''     
    new_df = df.copy()
    filepath = Path('NRC-Emotion-Lexicon','NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')
    
    emolex_df = pd.read_csv(filepath,
                            names=["word", "emotion", "association"],
                            sep='\t')
    emolex_words = emolex_df.pivot(index='word',
                                   columns='emotion',
                                   values='association').reset_index()
    #emotions = emolex_words.columns.drop('word')
    #emo_df = pd.DataFrame(0, index = sotudf.index, columns = emotions)    ## Let's replace that with the name of our dataframe ("sotudf")

    #new_df['emo_score'] = new_df[freq_lemmas_col].apply(emo_score_by_text)
    new_df['emo_score'] = new_df.apply(lambda x: emo_score_by_text(x[freq_lemmas_col], emolex_words), axis = 1)
    return(new_df)



15. Let's apply these functions to our corpus. Unfortunately, it takes 10-20 minutes to run this code on the whole corpus. So, let's just test it on the first three texts in the corpus:

In [ ]:
#sotusub2 = corpus_emo_scores(sotusub, "freqlemmas")          #this runs through the whole corpus. this takes 10-20 minutes
sotusub2 = corpus_emo_scores(sotusub.iloc[0:3], "freqlemmas")  #instead we can test it on just the first three texts
sotusub2.head()

<div class="alert alert-success" role="alert">
<p style="color:green">16. For each text in our corpus, this function retrieves scores for each of the 10 NRC categories (saved as a list in the "emo_score" column). It does so by passing the NRC Emotions Lexicon (saved as "emolex_words") and the column from our SOTU dataframe with the lemmas and their frequencies.</p>



17. We can split the list of emo_scores into separate columns for each emotion / sentiment using the code below:

In [ ]:
sotusub2[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']] = pd.DataFrame(sotusub2.emo_score.tolist(), index = sotusub2.index)
sotusub2 = sotusub2.drop(columns = ['emo_score'])
sotusub2.head(3)

It would be an interesting exercise to compare these automatically calculated NRC scores (derived from texts of these addresses) to scores given by humans who watched these addresses live. Obviously, sentiment analysis does not catch sarcasm, derision, and tone in general.

18. To save time running the sentiment analysis function across the entire dataframe, rather than just the three texts above, we can just load the results and examine them.

In [ ]:
#then load nrc sentiment analysis results
sotusub2=pd.read_csv("sotu_nrc_full.csv",encoding = "utf-8")   #these are the full results from the text_emotion function
sotusub2.head()

<div class="alert alert-success" role="alert">
<h3 style="color:green">Code Together</h3>

<p style="color:green">19. Before visualizing the sentiment analysis, it may be useful to review the functions we defined in Step 14 and ran in Step 15. Notice first, in Step 15, we called the function "corpus_emo_scores." That function called another function, "emo_score_by_text," which, in turn, called the function, "emo_score."  Review the documention, in triple quotes, for each function to understand what they do. </p>

<p style="color:green">Let's examine some parts of these functions. First, the outermost function, "corpus_emo_scores" reads in the NRC Emotion Lexicon (see below), which awards ones or zeroes to thousands of English words to indicate whether they invoke any of eight emotions (anger, anticipation, disgust, fear, joy, sadness, surprise, trust) or one of two sentiments (positive or negative).</p>

<p style="color:green">To get a better idea how the NRC Emotion Lexicon assigns scores, let's use the folder directory on the left side of JHub to navigate to the NRC Emotion Lexicon dataset. <i>What potential problems do you anticipate would result from such an approach to sentiment analysis?</i></p>

</div>

In [ ]:
filepath = Path('NRC-Emotion-Lexicon','NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')
    
emolex_df = pd.read_csv(filepath,
                            names=["word", "emotion", "association"],
                            sep='\t')
print("***emolex df: ",emolex_df.head())

<div class="alert alert-success" role="alert">
<p style="color:green">19b. As you can see the <b>.pivot()</b> method converts emolex_df from long to wide.</p>
</div>

In [ ]:
emolex_words = emolex_df.pivot(index='word',
                                   columns='emotion',
                                   values='association').reset_index()
emolex_words.head()

<div class="alert alert-success" role="alert">
<p style="color:green">19c. Then, it extracts a list of the 8 emotions / 2 sentimensts from this dataset
</p>
</div>

In [ ]:
emotions = emolex_words.columns.drop('word')
print("\n***list of emotions to be used as column names :",emotions)

<div class="alert alert-success" role="alert">
<p style="color:green">20. So, for example, if the address by Lincoln in 1863 uses the word "abandon", this code will retrieve the NRC scores for abandon: </p>

```
abandon	anger	0
abandon	anticipation	0
abandon	disgust	0
abandon	fear	1
abandon	joy	0
abandon	negative	1
abandon	positive	0
abandon	sadness	1
abandon	surprise	0
abandon	trust	0
```

<p style="color:green">It then will add 1 to the "fear", "sadness", and "negative" scores for the Lincoln 1863 address.</p>
</div>

</div>

22. Back to our dataframe of SOTU addresses and their respective emotion / sentiment scores. Frequently, in data science, calculating percentages or proportions is more meaningful than working with absolute numbers. Otherwise, our calculated emo scores may better reflect the length of each address than the actual emotional tenor of each given speech. 

Let's replace the absolute counts with each emotion's score divided by the number of tokens.

In [ ]:
emotions = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']
for emotion in emotions:
    sotusub2[emotion] = sotusub2[emotion] / sotusub2['numtoks']
sotusub2.head()

In [ ]:
sotusub2 = sotusub2.sort_values(by = "year")

23. Finally, we can plot these sentiment scores. Here we will use the **Matplotlib** library (plt).

In [ ]:
sotusub2['pres_year'] = sotusub2['pres']  + " " + sotusub2['year'].astype(str)
# length = sum([len(sotudf3['word_count'])])
x = sotusub2['year'].tolist()
for emotion in emotions:
    y = sotusub2[emotion]
    plt.plot(x,y,label = sotusub2['pres_year'])
    plt.title('{} sentiment of State of the Union Addresses'.format(emotion, fontsize=24))
    plt.xlabel('Year', fontsize=15)
    plt.ylabel('Average Sentiment', fontsize=15)
    plt.legend(loc='best', fontsize=15, bbox_to_anchor=(1.05, 1))
    plt.show()

24. We can also <b style="color:red">color</b><b style="color:blue">-code</b> <b style="color:green">this</b> <b style="color:orange">data</b> <b style="color:purple">by president</b>. We can use the **.shift()** and **.cumsum()** functions to label each administration and then the **groupby()** function to create a list of years for each successive presidential administration.

In [ ]:
sotusub2["presnum"] = (sotusub2["pres"] != sotusub2["pres"].shift()).cumsum()
presdf = sotusub2[["pres","year","presnum","pres_year"]]
presdf.to_csv("pres_sotulist.csv",encoding='utf-8')
presyears = presdf.groupby('presnum')['year'].apply(list)
print(presyears[-10:])

25. Now, we can modify our plot code from above to color-code each presidential administration and to add a line to show the moving average of each emotion or sentiment.

In [ ]:
def movingaverage(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

x = sotusub2['year'].tolist()
for emotion in emotions:
    fig,ax = plt.subplots()
    y = sotusub2[emotion]
    ctr=0
    plt.plot(x,y,label = sotusub2['pres_year'], linestyle = 'dashed', linewidth=2, color = 'gray')
    for presyrs in presyears:
        plt.plot(presyrs, y[ctr:ctr+len(presyrs)],label = sotusub2.iloc[ctr]['pres'],linewidth=2)
        ctr+=len(presyrs)
    plt.plot(x,movingaverage(y, 10), color='k', linewidth=2, linestyle=':', label = 'Moving Average')
    plt.title('{} sentiment of State of the Union Addresses'.format(emotion, fontsize=24))
    plt.xlabel('Year', fontsize=15)
    plt.ylabel('Average Sentiment', fontsize=15)
    plt.legend(loc='best', fontsize=7, bbox_to_anchor=(1.05, 1))
    image_format = "png"
    image_name = "sotu_%s_sentiment.%s"%(emotion,image_format)
    fig.savefig(image_name, format=image_format, dpi = 300)
    plt.show()

<div class="alert alert-info" role="alert" style='color:blue'><h3>Exercise for Part IV</h3>

<p>26. Copying and pasting some of the relevant code from above, create a single plot combining positive and negative sentiment scores to identify which presidents used the most emotional language.</p>
</div>